Задачи

Разработать модель чат-бота для ответов на вопросы

1) Реализовать подход “мешок слов”.

2) Реализовать подход TF-IDF.

3) Сравнить точность на тестовой выборке, которую создать
вручную.

In [ ]:
import scipy.spatial
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Perm_IT_Univer/ML/9_Text_Analis/sber_questions.xlsx', sheet_name='All', names=['question', 'answer'])
df = df.dropna()
df.head()

,question,answer
0,Как узнать условия кредитования?,"Узнайте подробные условия, по кредиту и рассчи..."
1,"Могу ли я взять кредит в Сбербанке, если у мен...","Да, вы можете получить в Сбербанке новый креди..."
2,Как подать заявку на кредит в Сбербанке?,Удобнее всего подать заявку на кредит в Сберба...
3,Как оформить кредит в Сбербанк Онлайн?,Чтобы подать заявку на кредит в Сбербанк Онлай...
4,Какой срок рассмотрения заявки на кредит в Сбе...,Максимальный срок рассмотрения заявки – 2 рабо...


In [ ]:
df['question'] = df['question'].str.lower()
df.head()

,question,answer
0,как узнать условия кредитования?,"Узнайте подробные условия, по кредиту и рассчи..."
1,"могу ли я взять кредит в сбербанке, если у мен...","Да, вы можете получить в Сбербанке новый креди..."
2,как подать заявку на кредит в сбербанке?,Удобнее всего подать заявку на кредит в Сберба...
3,как оформить кредит в сбербанк онлайн?,Чтобы подать заявку на кредит в Сбербанк Онлай...
4,какой срок рассмотрения заявки на кредит в сбе...,Максимальный срок рассмотрения заявки – 2 рабо...


In [ ]:
sentences = df['question'].tolist()
answers = df['answer'].tolist()

***Мешок слов***

In [ ]:
tokens = set(' '.join(sentences).split())
sentences_vectors = [[sentence.count(token) for token in tokens]
           for sentence in sentences]

In [ ]:
tokens

{'"сбербанка"',
 '"спасибо',
 '"спасибо"?',
 '(образовательной,',
 '(отличному',
 '1',
 '14',
 '18',
 '2019',
 '4%',
 '900',
 '«идут»',
 '«отдаю',
 '«подарок»,',
 '«спасибо',
 'а',
 'баланс',
 'банк',
 'банка,',
 'банка?',
 'банках?',
 'банке',
 'банке,',
 'банке?',
 'банком',
 'без',
 'безопасно',
 'бесконтактной',
 'бесконтактную',
 'бесконтактных',
 'близко',
 'больше',
 'больше,',
 'большое',
 'бонусами',
 'бонусами?',
 'бонусных',
 'бонусов',
 'бонусов?',
 'бонусы',
 'бонусы?',
 'брать',
 'будет',
 'будут',
 'был',
 'быстро',
 'в',
 'валюте',
 'верну/обменяю',
 'вернуть',
 'взять',
 'вижу',
 'вклад',
 'вклад,',
 'вклад.',
 'вклад?',
 'вклада',
 'вклада,',
 'вклада?',
 'вкладов?',
 'вкладом',
 'вкладу',
 'вкладу?',
 'вкладчик',
 'вкладчика?',
 'владельцем?',
 'внести',
 'вовремя?',
 'возможно?',
 'возраста',
 'воспользоваться',
 'времени',
 'временно',
 'все',
 'всех',
 'входит',
 'выбор',
 'выбор?',
 'выбрать',
 'выгодная?',
 'выдается',
 'выписку',
 'выполненных',
 'выполнять',
 

In [ ]:
def bag_of_words(new_sentence):

  new_sentence = new_sentence.lower()
  new_tokens = new_sentence.split()  
  new_sentences_vector = [new_tokens.count(token) for token in tokens]

  df = pd.DataFrame(columns=['sentence', 'distance'])
  for i, sentence in enumerate(sentences):
      df = df.append({
          'sentence': sentence,
          'distance': scipy.spatial.distance.cosine(new_sentences_vector, sentences_vectors[i]),
          'answer': answers[i]
      },ignore_index=True)
  return str(df.sort_values(by='distance').head(1)['answer'].values)

***TF-IDF***

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
len_vector = len(vectorizer.get_feature_names())
print([(i, word) for i, word in enumerate(vectorizer.get_feature_names())])
sentences_vectors_2 = [list(i) for i in csr_matrix(X).toarray()]
len_vector

[(0, '14'), (1, '18'), (2, '2019'), (3, '900'), (4, 'баланс'), (5, 'банк'), (6, 'банка'), (7, 'банках'), (8, 'банке'), (9, 'банком'), (10, 'без'), (11, 'безопасно'), (12, 'бесконтактной'), (13, 'бесконтактную'), (14, 'бесконтактных'), (15, 'близко'), (16, 'больше'), (17, 'большое'), (18, 'бонусами'), (19, 'бонусных'), (20, 'бонусов'), (21, 'бонусы'), (22, 'брать'), (23, 'будет'), (24, 'будут'), (25, 'был'), (26, 'быстро'), (27, 'валюте'), (28, 'верну'), (29, 'вернуть'), (30, 'взять'), (31, 'вижу'), (32, 'вклад'), (33, 'вклада'), (34, 'вкладов'), (35, 'вкладом'), (36, 'вкладу'), (37, 'вкладчик'), (38, 'вкладчика'), (39, 'владельцем'), (40, 'внести'), (41, 'вовремя'), (42, 'возможно'), (43, 'возраста'), (44, 'воспользоваться'), (45, 'времени'), (46, 'временно'), (47, 'все'), (48, 'всех'), (49, 'входит'), (50, 'выбор'), (51, 'выбрать'), (52, 'выгодная'), (53, 'выдается'), (54, 'выписку'), (55, 'выполненных'), (56, 'выполнять'), (57, 'выходной'), (58, 'вышла'), (59, 'где'), (60, 'годовое')

543

In [ ]:
def TF_IDF(new_sentence):
  new_sentence = new_sentence.lower()
  new_vector = csr_matrix(vectorizer.transform([new_sentence])).toarray()[0]
  df = pd.DataFrame(columns=['sentence', 'distance'])
  for i, sentence in enumerate(sentences):
      df = df.append({
          'sentence': sentence,
          'distance': scipy.spatial.distance.cosine(new_vector, sentences_vectors_2[i]),
          'answer': answers[i]
      },ignore_index=True)
  return str(df.sort_values(by='distance').head(1)['answer'].values)

In [ ]:
question = 'На какой срок даете кредит?'

print('Мешок слов:')
print(bag_of_words(question))
print('TF-IDF:')
print(TF_IDF(question))

Мешок слов:
['Если вы подавали заявку на кредит в Сбербанк Онлайн, посмотрите статус там же в разделе «Кредиты»,  или уточните в офисе банка. Когда заявку рассмотрят, вам придёт СМС с результатом']
TF-IDF:
['Максимальный срок рассмотрения заявки – 2 рабочих дня.']


In [ ]:
question = 'Что делать в случае утери карты?'

print('Мешок слов:')
print(bag_of_words(question))
print('TF-IDF:')
print(TF_IDF(question))

Мешок слов:
['Находясь на уровне Огромное Спасибо или уровне Больше, чем Спасибо вы можете подключить 1 или 2 категории на выбор. Расплачиваясь картой Сбербанка за покупки в торговых точках, соответствующих подключенной категории вы будете получать повышенные бонусы. Краткое описание категорий с повышенными бонусами на выбор указано ниже:\n\nКатегория "Электроника и Бытовая техника" позволит вам получать 3% бонусами за покупку цифровой, бытовой техники и электроники.\nКатегория "Досуг и Развлечения" позволит вам получать 10% бонусами при оплате в художественных галереях, боулинг-клубах, танцевальных и театральных студиях, музеях, парках аттракционов, зоопарках и других развлекательных местах.\nКатегория "Кино и Театр" позволит вам получать 10% бонусами за оплату билетов в кинотеатры, театры, музыкальные и другие мероприятия.\nКатегория "Такси и лимузины" позволит вам получать 10% бонусами за оплату сервисов такси и каршеринга.\nКатегория "Салоны красоты" позволит вам получать 10% бонус

In [ ]:
question = 'Сколько длиться рассмотрение заявки на ипотечный кредит?'

print('Мешок слов:')
print(bag_of_words(question))
print('TF-IDF:')
print(TF_IDF(question))

Мешок слов:
['Если вы подавали заявку на кредит в Сбербанк Онлайн, посмотрите статус там же в разделе «Кредиты»,  или уточните в офисе банка. Когда заявку рассмотрят, вам придёт СМС с результатом']
TF-IDF:
['Максимальный срок рассмотрения заявки – 2 рабочих дня.']
